In [57]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [59]:
print(tf.config.list_physical_devices('GPU'))

[]


In [58]:
!pip install keras

In [10]:
(X_train_full,y_train_full),(X_test,y_test)= tf.keras.datasets.fashion_mnist.load_data()
X_train_full= X_train_full/255.0
X_test=X_test/255.0
X_valid,X_train=X_train_full[:5000],X_train_full[5000:]
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS=[tf.keras.layers.Flatten(input_shape=[28,28]),
       tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
       tf.keras.layers.LeakyReLU(),
       tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
       tf.keras.layers.LeakyReLU(),
       tf.keras.layers.Dense(10,activation="softmax")]

model=tf.keras.models.Sequential(LAYERS)

In [12]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer= tf.keras.optimizers.SGD(learning_rate=1e-3),
             metrics=["accuracy"])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 26661

In [17]:
history=model.fit(X_train,y_train, epochs=10,
                 validation_data=(X_valid,y_valid),verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 0.4870 - accuracy: 0.8339 - val_loss: 0.4772 - val_accuracy: 0.8414 - 6s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.4789 - accuracy: 0.8358 - val_loss: 0.4695 - val_accuracy: 0.8436 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 7s - loss: 0.4716 - accuracy: 0.8372 - val_loss: 0.4725 - val_accuracy: 0.8378 - 7s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 7s - loss: 0.4654 - accuracy: 0.8397 - val_loss: 0.4602 - val_accuracy: 0.8470 - 7s/epoch - 4ms/step
Epoch 5/10
1719/1719 - 7s - loss: 0.4600 - accuracy: 0.8417 - val_loss: 0.4530 - val_accuracy: 0.8490 - 7s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 7s - loss: 0.4546 - accuracy: 0.8435 - val_loss: 0.4488 - val_accuracy: 0.8498 - 7s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 8s - loss: 0.4498 - accuracy: 0.8448 - val_loss: 0.4439 - val_accuracy: 0.8538 - 8s/epoch - 4ms/step
Epoch 8/10
1719/1719 - 6s - loss: 0.4456 - accuracy: 0.8464 - val_loss: 0.4474 - val_accuracy: 0.8488 - 6s/epoch - 4ms/step
Epoch 9/

In [18]:
model.save("pretrained_mnist_model.h5")

C:\Users\arvin\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#  Transfer learning

# New Problem Statement

Classify handwritten didgits into odd and even

In [19]:
pretrained_mnist_model= tf.keras.models.load_model("pretrained_mnist_model.h5")

In [20]:
pretrained_mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 26661

In [23]:
for layer in pretrained_mnist_model.layers:
    print(f"{layer.name}:{layer.trainable}")

flatten_1:True
dense_3:True
leaky_re_lu_2:True
dense_4:True
leaky_re_lu_3:True
dense_5:True


In [24]:
for layer in pretrained_mnist_model.layers[:-1]:  # Leaves the last layer unfreezed  that means last layer is trainable
    layer.trainable= False # process is called Freezing the layer getting trained
    print(f"{layer.name}:{layer.trainable}")

flatten_1:False
dense_3:False
leaky_re_lu_2:False
dense_4:False
leaky_re_lu_3:False


In [25]:
for layer in pretrained_mnist_model.layers:
    print(f"{layer.name}:{layer.trainable}")

flatten_1:False
dense_3:False
leaky_re_lu_2:False
dense_4:False
leaky_re_lu_3:False
dense_5:True


In [27]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]
'''LAYERS_TL=[
    lower_pretrained_layers,
    tf.keras.layers.Dense(2,activation="softmax")    
]
'''
new_model= tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(2,activation="softmax") 
)

In [28]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265802 (1.01 MB)
Trainable params: 202 (

In [32]:
100 * 2 + 2 # trainable parameter

202

In [33]:
266610-1010 # Non trainable parameter

265600

In [34]:
265600 + 202  #Total Parameter

265802

In [35]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [36]:
9%2 

1

In [37]:
6%2

0

In [38]:
np.where(y_train%2==0, 1,0)

array([1, 1, 0, ..., 0, 1, 0])

In [41]:
y_train[0], y_train[-1]

(4, 5)

In [42]:
def update_even_odd_labels(labels):
    for idx,label in enumerate(labels):
        labels[idx] = np.where(label % 2 ==0 , 1, 0)
    return labels

In [48]:
y_train_binary, y_test_binary, y_valid_binary = update_even_odd_labels([y_train,y_test, y_valid])

In [49]:
np.unique(y_train_binary)

array([0, 1])

In [50]:
new_model.compile(loss="sparse_categorical_crossentropy",
             optimizer= tf.keras.optimizers.SGD(learning_rate=1e-3),
             metrics=["accuracy"])

In [51]:
history=new_model.fit(X_train,y_train_binary, epochs=10,
                 validation_data=(X_valid,y_valid_binary),verbose=2)

Epoch 1/10
1719/1719 - 8s - loss: 0.2328 - accuracy: 0.9197 - val_loss: 0.1587 - val_accuracy: 0.9506 - 8s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 7s - loss: 0.1481 - accuracy: 0.9510 - val_loss: 0.1411 - val_accuracy: 0.9546 - 7s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 8s - loss: 0.1365 - accuracy: 0.9545 - val_loss: 0.1341 - val_accuracy: 0.9562 - 8s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 6s - loss: 0.1308 - accuracy: 0.9558 - val_loss: 0.1303 - val_accuracy: 0.9570 - 6s/epoch - 4ms/step
Epoch 5/10
1719/1719 - 7s - loss: 0.1274 - accuracy: 0.9567 - val_loss: 0.1276 - val_accuracy: 0.9582 - 7s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 7s - loss: 0.1249 - accuracy: 0.9573 - val_loss: 0.1255 - val_accuracy: 0.9596 - 7s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.1229 - accuracy: 0.9577 - val_loss: 0.1240 - val_accuracy: 0.9600 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 6s - loss: 0.1216 - accuracy: 0.9583 - val_loss: 0.1229 - val_accuracy: 0.9596 - 6s/epoch - 4ms/step
Epoch 9/

In [52]:
new_model.evaluate(X_test, y_test_binary)

313/313 [==============================] - 1s 2ms/step - loss: 0.1299 - accuracy: 0.9577


[0.12986969947814941, 0.9577000141143799]

In [53]:
X_new =X_test[:3]
y_test[:3], y_test_binary[:3]

(array([9, 2, 1], dtype=uint8), array([0, 1, 0]))

In [54]:
np.argmax(new_model.predict(X_new),axis=-1)

1/1 [==============================] - 0s 113ms/step


array([0, 1, 0], dtype=int64)